In [ ]:
import xarray as xr
import numpy as np
import rioxarray
import h5netcdf
import h5py
import netCDF4
import scipy.ndimage

import hvplot.xarray
import geoviews as gv

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
surface_velocity = xr.open_dataset('data/GRE_G0120_0000.nc').coarsen(x=100, y=100, boundary='trim').mean()
surface_velocity_spacing_m = np.abs(np.median(np.diff(surface_velocity['x'])))
assert(np.abs(np.max(np.diff(surface_velocity['x']))) == surface_velocity_spacing_m)
assert(np.abs(np.min(np.diff(surface_velocity['x']))) == surface_velocity_spacing_m)
assert(np.abs(np.max(np.diff(surface_velocity['y']))) == surface_velocity_spacing_m)
assert(np.abs(np.min(np.diff(surface_velocity['y']))) == surface_velocity_spacing_m)

age_model = xr.open_dataset('data/RRRAG4_Greenland_1993_2013_01_age_grid.nc',
                               engine='h5netcdf').rename(
                                   {'number of grid points in x-direction': 'x', 'number of grid points in y-direction': 'y', 'number of vertical layers': 'layers', 'number of isochrones': 'original_isochrones',
                                    'x': 'x_km', 'y': 'y_km'}
                                   )
# Convert km to meters for consistency
age_model = age_model.assign_coords({'x':age_model.x_km[:,0]*1000, 'y':age_model.y_km[0,:]*1000})

bedmachine = xr.open_dataset('data/BedMachineGreenland-v5.nc')
s = bedmachine['surface'].interp(x=surface_velocity['x'], y=surface_velocity['y'])

bedmachine_spacing_m = np.abs(np.median(np.diff(bedmachine['x'])))
assert(np.abs(np.max(np.diff(bedmachine['x']))) == bedmachine_spacing_m)
assert(np.abs(np.min(np.diff(bedmachine['x']))) == bedmachine_spacing_m)
assert(np.abs(np.max(np.diff(bedmachine['y']))) == bedmachine_spacing_m)
assert(np.abs(np.min(np.diff(bedmachine['y']))) == bedmachine_spacing_m)

In [ ]:
surface_velocity

In [ ]:
crs_3413 = ccrs.Stereographic(central_latitude=90, central_longitude=-45, true_scale_latitude=70)

def plot_on_map(da, **kwargs):
    if 'cmap' not in kwargs:
        if da.min() >= 0:
            kwargs['cmap'] = 'OrRd'
        else:
            kwargs['cmap'] = 'RdBu_r'
            if 'clim' not in kwargs:
                abs_max = float((np.abs(da)).max())
                kwargs['clim'] = (-abs_max, abs_max)

    plot = da.hvplot.image(x='x', y='y', geo=True, rasterize=True, crs=crs_3413, **kwargs)
    plot = plot * gv.feature.coastline()
    return plot

In [ ]:
smoothing_kernel_std = 10000 # m

us_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, surface_velocity['vx'], kwargs={'sigma': smoothing_kernel_std / surface_velocity_spacing_m, 'truncate': 2})
vs_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, surface_velocity['vy'], kwargs={'sigma': smoothing_kernel_std / surface_velocity_spacing_m, 'truncate': 2})
s_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, s, kwargs={'sigma': smoothing_kernel_std / surface_velocity_spacing_m, 'truncate': 2})

In [ ]:
s_dx = s_filt.differentiate('x')
s_dy = s_filt.differentiate('y')
abs_grad_s = np.sqrt(s_dx**2 + s_dy**2)

rho=918.0
g=9.8
n=1.5
A0_n3 = 3.985e-13 # s^-1 Pa^-3
Q = 60.0e3 # J mol^-1
R = 8.314 # J mol^-1 K^-1
T_rel_p = 273.15 - 20

# Adjust A0 for match n
log_ref_stress = 6.0
log_ref_strain_rate = np.log10(2*A0_n3) + 3 * log_ref_stress
log_2A = log_ref_strain_rate - n * log_ref_stress # log10(2A)
A0 = 10**(log_2A) / 2

A = A0 * np.exp(-Q/(R * T_rel_p)) * (365*60*60) # convert s^-1 to yr^-1

# def sia_u(x, y, z, rho=918.0, g=9.8, n=3, A=A):
#     us = us_filt.interp(x=x, y=y)
#     u_diff_from_surf = 2 * (rho * g)**n * abs_grad_h.interp(x=x, y=y)**(n-1) * h_dx.interp(x=x, y=y) * A * (h.interp(x=x,y=y) - z)**(n+1) / (n+1)
#     return us + u_diff_from_surf


In [ ]:
s_minus_z = s * 0.7
u_at_depth = us_filt + (2 * (rho * g)**n * abs_grad_s**(n-1) * s_dx * A * (s_minus_z)**(n+1) / (n+1))
v_at_depth = vs_filt + (2 * (rho * g)**n * abs_grad_s**(n-1) * s_dy * A * (s_minus_z)**(n+1) / (n+1))

du_dz_at_depth = -2 * A * (rho * g * (s_minus_z))**n * abs_grad_s**(n-1) * s_dx
dv_dz_at_depth = -2 * A * (rho * g * (s_minus_z))**n * abs_grad_s**(n-1) * s_dy

# Sanity check code:
# u_at_depth_offset = us_filt + (2 * (rho * g)**n * abs_grad_s**(n-1) * s_dx * A * ((s * 0.5)+100)**(n+1) / (n+1))
# v_at_depth_offset = vs_filt + (2 * (rho * g)**n * abs_grad_s**(n-1) * s_dy * A * ((s * 0.5)+100)**(n+1) / (n+1))

# du_dz_approx = (u_at_depth - u_at_depth_offset) / 100
# dv_dz_approx = (v_at_depth - v_at_depth_offset) / 100

# plot_on_map(du_dx_approx - du_dx_at_depth)

In [ ]:
# Parts of crazy equation from notes
pA_x = us_filt.differentiate('x')
pA_y = vs_filt.differentiate('y')

pB = 2*A*(rho*g)**n * s_minus_z**n * abs_grad_s**(n-1) / (n+1)

pC_x = (n+1)*(s_dx**2)
pC_y = (n+1)*(s_dy**2)

pD_x = s_minus_z * s_dx.differentiate('x')
pD_y = s_minus_z * s_dy.differentiate('y')

pE_x = s_minus_z * (n-1) * (s_dx)**2 * abs_grad_s**(-1) * abs_grad_s.differentiate('x') # TODO: Is it abs_grad_s.differentiate(x) or s_dx.differentiate(x) ?
pE_y = s_minus_z * (n-1) * (s_dy)**2 * abs_grad_s**(-1) * abs_grad_s.differentiate('y')

du_dx = pA_x + pB * (pC_x + pD_x + pE_x)
dv_dy = pA_y + pB * (pC_y + pD_y + pE_y)
dw_dz_at_depth = -1 * (du_dx + dv_dy)

In [ ]:
plot_on_map(dw_dz_at_depth, clim=(-1e-3, 1e-3))

In [ ]:
plot_on_map(du_dz_at_depth, clim=(-1e-3, 1e-3))

In [ ]:
plot_on_map(np.abs(du_dz_at_depth / dw_dz_at_depth), clim=(0, 10))

In [ ]:
max_layer_slope = np.arctan(np.abs(dw_dz_at_depth) * 0.1 / np.abs(du_along_flow_dz))
plot_on_map(max_layer_slope * (180 / np.pi), clim=(0, 0.2))

In [ ]:
plot_on_map(np.arctan(tan_layer_slope), clim=(0, 0.2))

In [ ]:
#age_model

In [ ]:
age_model['age_iso'].loc[{'original_isochrones': 2}].to_numpy()

In [ ]:
plot_on_map(age_model['depth_iso'].loc[{'original_isochrones': 0}], clim=(0, 3000))

In [ ]:
sel_layer = age_model['depth_iso'].loc[{'original_isochrones': 0}]
layer_dx = sel_layer.differentiate('x')
layer_dy = sel_layer.differentiate('y')

tan_layer_slope = (np.sqrt(layer_dx**2+layer_dy**2)).interp(x=us_filt['x'], y=us_filt['y'])

plot_on_map(tan_layer_slope, clim=(0, 0.05))

In [ ]:
du_along_flow_dz = np.sqrt(du_dz_at_depth**2 + dv_dz_at_depth**2)
layer_signal = np.abs(tan_layer_slope * du_along_flow_dz)
vertical_strain_rate_signal = np.abs(dw_dz_at_depth)

plot_on_map(layer_signal / (layer_signal + vertical_strain_rate_signal), clim=(0, 1), frame_height=500)

In [ ]:
layer_elevation_filt.differentiate("x")

In [ ]:
flow_angle_at_depth = np.arctan2(v_at_depth, u_at_depth)
flow_angle_at_surface = np.arctan2(vs_filt, us_filt)
plot_on_map(flow_angle_at_depth - flow_angle_at_surface, clim=(-np.pi, np.pi))

In [ ]:
surf_vel_mag = np.sqrt(us_filt**2 + vs_filt**2)
vel_at_depth_mag = np.sqrt(u_at_depth**2 + v_at_depth**2)
plot_on_map(surf_vel_mag - vel_at_depth_mag, clim=(-2,2), frame_height=500)
#plot_on_map(u_at_depth_diff, clim=(-2,2), frame_height=500)

In [ ]:
vel_mag_diff = (surf_vel_mag - vel_at_depth_mag)

bin_edges = np.arange(-2, 10.1, 0.1)
p = vel_mag_diff.plot.hist(bins=bin_edges, xlim=(-5, 10))
plt.show()


In [ ]:
plot_on_map(abs_grad_h * 1000, frame_height=500, clim=(-10, 10))

In [ ]:
plot_on_map(np.abs(us_filt) - np.abs(u_at_depth), height=500, clim=(-1, 1))

In [ ]:
plot_on_map(us_filt, height=500, clim=(-100, 100))

In [ ]:
us_filt.interp(x=-1.067e5, y=-2.267e6)

In [ ]:
[sia_u(-1.067e5, -2.267e6, z=z) for z in np.arange(0, 1000, 100)]

In [ ]:
A

In [ ]:
rho=918.0
g=9.8
n=3
x = -1.067e5
y = -2.267e6
#z = h.interp(x=x, y=y)
#2 * (rho * g)**n * abs_grad_h.interp(x=x, y=y)**(n-1) * h_dx.interp(x=x, y=y) * (A*60*60*365) * (h.interp(x=x,y=y) - z)**(n+1) / (n+1)

In [ ]:
plot_on_map(us_filt, height=500)

In [ ]:
def plot_on_map(da, **kwargs):
    if 'cmap' not in kwargs:
        if da.min() >= 0:
            kwargs['cmap'] = 'OrRd'
        else:
            kwargs['cmap'] = 'RdBu_r'
            if 'clim' not in kwargs:
                abs_max = float((np.abs(da)).max())
                kwargs['clim'] = (-abs_max, abs_max)

    plot = da.hvplot.image(x='x', y='y', geo=True, rasterize=True, crs=crs_3413, **kwargs)
    plot = plot * gv.feature.coastline()
    return plot

plot_on_map(h, height=500, title='Ice Thickness [m]')